# crewAI Chessmaster

In [1]:
import chess
import chess.engine

## Assemble the crew for a chess game

In [20]:
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task

# Uncomment the following line to use an example of a custom tool
# from chess.tools.custom_tool import MyCustomTool

# Check our tools documentations for more information on how to use them
# from crewai_tools import SerperDevTool

class ChessCrew:
	""" ChessCrew class is responsible for creating a Crew of Agents and Tasks to predict the best next move in a chess game.
		Attributes:
			llm (object): Language model used by the Agents.
			tools (list, optional): List of tools to be used by the Agents. Defaults to None.
	"""
	def __init__(self, llm, tools=None):
		self.tools = tools or []
		self.llm = llm
		self.task_description = """Conduct a thorough analysis of board state {board} and moves history so far {moves}
						\n If needed, search the internet with query ```what's the best move for {board}```\n
						Predict the best next move
						Here is the list of legal move:{legal_moves}\n
						Here is the feedback {feedback}""" if tools else """Conduct a thorough analysis of board state {board} and moves history so far {moves}
						Predict the best next move
						Here is the list of legal move:{legal_moves}\n
						Here is the feedback {feedback}"""
		self.chess_master = Agent(
					role='Chess Grandmaster',
					goal='Predict best next move to win the chess game',
					backstory="""As one of the greatest chess players of all time, 
						Known for your deep strategic understanding, innovative approaches, and fierce competitive spirit.""",
					tools=self.tools, # Example of custom tool, loaded on the beginning of file
					llm=self.llm,
					verbose=True
				)
		self.next_move = Task(
					description=self.task_description,
					expected_output='The best next move. This best move can be anyone in the legal move list not just the first one. And only output the move, nothing else.',
					agent=self.chess_master
				)
		self.crew = Crew(
					agents=[self.chess_master], # Automatically created by XXX @agent decorator
					tasks=[self.next_move], # Automatically created by XXX @task decorator
					process=Process.sequential,
					verbose=2,
					# process=Process.hierarchical, # In case you wanna use that instead XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
				)

## Chess Game
### Parameters
`ai_player`
 (optional, default is 'llm'): This parameter specifies the type of AI player you want to play against.


`board`
 (optional, default is 
None
): This parameter allows you to provide an initial chess board state in FEN.

`engine_config`
 (optional, default is 
None
): This parameter is used to configure the chess engine (Stockfish) if 
ai_player
 is set to 'engine'.

`tools` (optional, default is 
None
): This parameter allows you to utilize an ai llm agent with tools.




In [3]:
class ChessGame:
    # players can be llm, engine, user
    def __init__(self, ai_player='llm', llm=None, board=None, engine_config=None, tools=None):
        self.ai_player = ai_player
        self.tools = tools
        self.llm = llm
        if engine_config is not None:
            # engine_config = {"path": "/opt/homebrew/Cellar/stockfish/16/bin/stockfish", "skill_lv": 0}
            # Initialize the chess engine
            self.engine = chess.engine.SimpleEngine.popen_uci(engine_config['path'])
            # Configure Stockfish to a specific skill level
            self.engine.configure({"Skill Level": 0})

        # Initialize a chess board
        self.board = board or chess.Board()
        self.moves = []

    def get_agent_move(self):
        feedback = ""
        board = self.board
        while True:
            # Replace with your inputs, it will automatically interpolate any tasks and agents information
            inputs = {
                'board': board.fen(),
                'moves': self.moves,
                'legal_moves': ",".join(
                    [str(_move) for _move in board.legal_moves]),
                'feedback': feedback,
            }
            chess_crew = ChessCrew(self.llm, self.tools).crew
            _result = chess_crew.kickoff(inputs=inputs)
            next_move = _result

            try:
                _move = chess.Move.from_uci(next_move)

                if _move in board.legal_moves:
                    return _move
                else:
                    feedback = f"Agent's generated move {_move} is not valid currently."
            except Exception as e:
                feedback = "Failed to parse the Agent's generated move. Retrying..."

    def next(self, user_input=None):
        board = self.board
        move = None
        if user_input:
            try:
                move = chess.Move.from_uci(user_input)
            except:
                print("Invalid move, please try again")
        elif self.ai_player == 'llm':
            move = self.get_agent_move()
        elif self.ai_player == 'engine':
            result = self.engine.play(board, chess.engine.Limit(time=0.001))
            move = result.move
            

        self.board.push(move)
        self.moves.append(move.uci())  # Store UCI move in the list

        return move

    def show_board(self):
        return print(self.board)
    

    def auto_play(self):
        board = self.board
        while not board.is_game_over():
            move = self.next()
            print(f'{"White" if board.turn else "Black"} made a move: {move.uci()}')

        # Check the result of the game
        winner = ""
        if board.is_checkmate():
            if board.turn:
                winner = "Black"
            else:
                winner = "White"
        elif board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition() or board.is_variant_draw():
            winner = "Draw"

        if winner == "Black":
            return "Black wins by checkmate."
        elif winner == "White":
            return "White wins by checkmate."
        else:
            return "The game is a draw."

In [4]:
from langchain_community.chat_models import ChatOllama
# from langchain_experimental.llms.ollama_functions import OllamaFunctions

llm = ChatOllama(model="llama3.1", temperature=0.1)
# llm = OllamaFunctions(model="llama3.1", format="json")

In [5]:
chess_master = ChessGame(ai_player='llm', llm=llm)

In [6]:
chess_master.next("e2e3")

Move.from_uci('e2e3')

In [7]:
chess_master.show_board()

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . P . . .
P P P P . P P P
R N B Q K B N R


In [8]:
chess_master.next()

 [2024-08-01 14:38:53][DEBUG]: == Working Agent: Chess Grandmaster
 [2024-08-01 14:38:53][INFO]: == Starting Task: Conduct a thorough analysis of board state rnbqkbnr/pppppppp/8/8/8/4P3/PPPP1PPP/RNBQKBNR b KQkq - 0 1 and moves history so far ['e2e3']
						Predict the best next move
						Here is the list of legal move:g8h6,g8f6,b8c6,b8a6,h7h6,g7g6,f7f6,e7e6,d7d6,c7c6,b7b6,a7a6,h7h5,g7g5,f7f5,e7e5,d7d5,c7c5,b7b5,a7a5

						Here is the feedback 


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer: e7e6

> Finished chain.
 [2024-08-01 14:38:56][DEBUG]: == [Chess Grandmaster] Task output: e7e6




Move.from_uci('e7e6')

In [9]:
chess_master.show_board()

r n b q k b n r
p p p p . p p p
. . . . p . . .
. . . . . . . .
. . . . . . . .
. . . . P . . .
P P P P . P P P
R N B Q K B N R


## Using Search Tool

In [10]:
# !pip install -U langchain-community tavily-python

from langchain_community.tools.tavily_search import TavilySearchResults
# Setup environment

import os
import getpass

def _set_if_undefined(var: str) -> None:
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var)

_set_if_undefined("TAVILY_API_KEY")
# if _set_if_undefined not working, assign api key directory to the environment as follows.
# os.environ["TAVILY_API_KEY"] = "[YOUR TAVILY API KEY HERE]"

tavily_tool = TavilySearchResults()


In [10]:
from langchain_community.tools.tavily_search import TavilySearchResults
# Setup environment

import os

tavily_tool = TavilySearchResults()

In [11]:

chess_master = ChessGame(ai_player='llm', llm=llm, tools=[tavily_tool])

In [12]:
chess_master.next("e2e3")

Move.from_uci('e2e3')

In [13]:
chess_master.show_board()

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . P . . .
P P P P . P P P
R N B Q K B N R


In [14]:
chess_master.next()

2024-08-01 14:39:11,781 - 8641229312 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-01 14:39:11][DEBUG]: == Working Agent: Chess Grandmaster
 [2024-08-01 14:39:11][INFO]: == Starting Task: Conduct a thorough analysis of board state rnbqkbnr/pppppppp/8/8/8/4P3/PPPP1PPP/RNBQKBNR b KQkq - 0 1 and moves history so far ['e2e3']
						
 If needed, search the internet with query ```what's the best move for rnbqkbnr/pppppppp/8/8/8/4P3/PPPP1PPP/RNBQKBNR b KQkq - 0 1```

						Predict the best next move
						Here is the list of legal move:g8h6,g8f6,b8c6,b8a6,h7h6,g7g6,f7f6,e7e6,d7d6,c7c6,b7b6,a7a6,h7h5,g7g5,f7f5,e7e5,d7d5,c7c5,b7b5,a7a5

						Here is the feedback 


> Entering new CrewAgentExecutor chain...
Action: tavily_search_results_json
Action Input: { "query": "what's the best move for rnbqkbnr/pppppppp/8/8/8/4P3/PPPP1PPP/RNBQKBNR b KQkq - 0 1" 

[{'url': 'https://chess.stackexchange.com/questions/41016/uci-play-colors', 'content': 'Nov 17, 2022 · In startpos, the side to move is white. The startpos is just equal to a fen: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RN

Move.from_uci('e7e5')

In [15]:
chess_master.show_board()

r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . . . . . .
. . . . P . . .
P P P P . P P P
R N B Q K B N R
